In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 87 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595754 sha256=cf23d35fb5fcdd970302e13ce9dfb276a2de7a942fe3674601f24668f0f1ffa5
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 769 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 3.0 MB 34.9 MB/s 


In [91]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import urllib.request

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import bz2
from collections import Counter
import re
import nltk


#import seaborn as sns
from sklearn.metrics import roc_curve
from keras import backend as K


In [92]:
#데이터 불러오기
data = pd.read_csv('total_final_knu_사전점수_0429.csv')

In [93]:
data.head()

,Unnamed: 0,Unnamed: 0.1,reviews,label,new_label
0,0,0,﻿좋다 가격 구매 수납 공간 조금 많이 작다 정도 그냥 없다 모델 살 걸 그렇다 라...,0,0
1,1,1,침대 프레임 정말 실망 기존 싸다 제품 비교 상품 설명 보고 수납 달리 홀 뚫리다 ...,0,0
2,2,2,별로 사진 처럼 침대 하단 부 시트 지 가다 조금 벗겨지다 기사 설치 그렇다 건 지...,0,0
3,3,3,설치 기사 가다 프레임 헤드 쪽 깨다 못 보신 건지다 말씀 안 해주다 속상하다 누우...,0,0
4,4,4,월일 설치 달 사용 서랍 장이 분리 버리다 서랍 장이 톱밥 뭉치다 합판 너무 약하다...,0,0


In [94]:
data = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'label'], axis=1)

In [95]:
data

,reviews,new_label
0,﻿좋다 가격 구매 수납 공간 조금 많이 작다 정도 그냥 없다 모델 살 걸 그렇다 라...,0
1,침대 프레임 정말 실망 기존 싸다 제품 비교 상품 설명 보고 수납 달리 홀 뚫리다 ...,0
2,별로 사진 처럼 침대 하단 부 시트 지 가다 조금 벗겨지다 기사 설치 그렇다 건 지...,0
3,설치 기사 가다 프레임 헤드 쪽 깨다 못 보신 건지다 말씀 안 해주다 속상하다 누우...,0
4,월일 설치 달 사용 서랍 장이 분리 버리다 서랍 장이 톱밥 뭉치다 합판 너무 약하다...,0
...,...,...
20929,폭 아쉽다 기다 하지만 튼튼하다 분위기,0
20930,사용 좋다,1
20931,튼튼하다 수납 공간 좋다 여전하다 만족하다,1
20932,지방 보다 배송 늦다 기사 바쁘다 그냥 집 두다 가시 보다 균형 잡다 애 먹다 후 ...,0


In [96]:
#train & test 데이터로 나누기                                                 
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)

In [97]:
# 데이터 수 확인
print(len(train_data))
print(len(test_data))

16747
4187


In [98]:
!pip install konlpy
from konlpy.tag import Okt

okt = Okt()

In [99]:
X_train = []
for sentence in tqdm(train_data['reviews']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    X_train.append(tokenized_sentence)

100%|██████████| 16747/16747 [01:13<00:00, 228.70it/s]


In [100]:
print(X_train[23])

['전체', '적', '만족하다', '배송', '너무', '느리다', '힘들다', '본사', '색상', '잘못', '내다', '보내다', '더', '오래', '걸리다']


In [101]:
X_test = []
for sentence in tqdm(test_data['reviews']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    X_test.append(tokenized_sentence)

100%|██████████| 4187/4187 [00:19<00:00, 219.69it/s]


In [102]:
print(X_test[23])

['조립', '상당하다', '힘들다', '만화책', '꽂다', '유연하다', '상용', '중']


In [103]:
y_train = np.array(train_data['new_label'])
y_test = np.array(test_data['new_label'])

In [104]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [105]:
print(tokenizer.word_index)

{'좋다': 1, '배송': 2, '보다': 3, '너무': 4, '같다': 5, '안': 6, '가격': 7, '않다': 8, '생각': 9, '다': 10, '기사': 11, '설치': 12, '받다': 13, '제품': 14, '사용': 15, '없다': 16, '구매': 17, '오다': 18, '쓰다': 19, '들다': 20, '더': 21, '만족하다': 22, '그냥': 23, '만': 24, '많이': 25, '깔끔하다': 26, '부분': 27, '튼튼하다': 28, '해주다': 29, '괜찮다': 30, '친절하다': 31, '조립': 32, '주문': 33, '디자인': 34, '냄새': 35, '아니다': 36, '소파': 37, '대비': 38, '맘': 39, '걸리다': 40, '예쁘다': 41, '상품': 42, '수': 43, '그렇다': 44, '맞다': 45, '가다': 46, '분': 47, '수납': 48, '조금': 49, '색상': 50, '크다': 51, '정말': 52, '집': 53, '저렴하다': 54, '쿠션': 55, '정도': 56, '편하다': 57, '마음': 58, '못': 59, '침대': 60, '보이': 61, '듯': 62, '서랍': 63, '서다': 64, '식탁': 65, '사이즈': 66, '성비': 67, '쓸다': 68, '아쉽다': 69, '시간': 70, '주다': 71, '니': 72, '사진': 73, '다리': 74, '배송비': 75, '빠르다': 76, '인': 77, '앉다': 78, '이쁘다': 79, '가구': 80, '느낌': 81, '공간': 82, '마감': 83, '지다': 84, '아주': 85, '데': 86, '많다': 87, '원': 88, '적': 89, '걸': 90, '건': 91, '문': 92, '개': 93, '싸다': 94, '그리고': 95, '면': 96, '배': 97, '오래': 98, '장': 99, '빼다': 100, '달': 

In [106]:
total_cnt = len(tokenizer.word_index)

In [107]:
total_cnt

10085

In [108]:
vocab_size = total_cnt + 1

In [109]:
vocab_size

10086

In [110]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train) ###
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [111]:
print(X_train[:3])

[[48, 82, 323, 189, 148, 26], [230, 48, 43, 647, 170, 90, 15, 215, 1121, 33, 60, 238, 3144, 497, 173, 241, 500, 49, 24, 406, 283, 294, 3144, 2018, 965, 64, 1202, 109, 756, 459, 60, 128, 343, 1432, 1307, 118, 882, 227, 234, 219, 25, 604, 60, 402, 118, 234, 219, 509, 6, 255], [828, 1471, 1225, 868, 110, 16, 1, 95, 161, 149, 3530, 43, 571]]


In [112]:
max_len = 100

In [113]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [114]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [115]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

In [116]:
lstm = Bidirectional(LSTM(64, return_sequences = True))(embedded_sequences)

In [117]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [118]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, return_sequences=True, return_state=True))(lstm)

In [119]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 100, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [120]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

In [121]:
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

In [122]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [123]:
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [124]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', recall, precision, f1score])

In [125]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data=(X_test, y_test), verbose=1, callbacks=[es, mc])

Epoch 1/10
524/524 [==============================] - 101s 192ms/step - loss: 0.3729 - accuracy: 0.8372 - recall: 0.8063 - precision: 0.7953 - f1score: 0.7939 - val_loss: 0.2081 - val_accuracy: 0.9209 - val_recall: 0.9247 - val_precision: 0.9081 - val_f1score: 0.9138
Epoch 2/10
524/524 [==============================] - 90s 172ms/step - loss: 0.1687 - accuracy: 0.9366 - recall: 0.9352 - precision: 0.9340 - f1score: 0.9320 - val_loss: 0.1958 - val_accuracy: 0.9214 - val_recall: 0.9309 - val_precision: 0.9018 - val_f1score: 0.9134
Epoch 3/10
524/524 [==============================] - 90s 172ms/step - loss: 0.1068 - accuracy: 0.9617 - recall: 0.9598 - precision: 0.9605 - f1score: 0.9584 - val_loss: 0.2002 - val_accuracy: 0.9286 - val_recall: 0.9176 - val_precision: 0.9281 - val_f1score: 0.9200
Epoch 4/10
524/524 [==============================] - 90s 171ms/step - loss: 0.0767 - accuracy: 0.9734 - recall: 0.9739 - precision: 0.9713 - f1score: 0.9715 - val_loss: 0.2273 - val_accuracy: 0.929

In [126]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

131/131 [==============================] - 5s 39ms/step - loss: 0.2273 - accuracy: 0.9291 - recall: 0.9116 - precision: 0.9354 - f1score: 0.9207

 테스트 정확도: 0.9291


In [ ]:
f = open("/content/불용어사전_최종.txt", 'r',encoding='cp949')
line = f.readlines()
bul = []dlf
for word in line:
    word = word.replace('\n','')
    bul.append(word)
print(bul)

['가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공동으로', '과', '과연', '관계가 있다', '관계없이', '관련이 있다', '관하여', '관한', '관해서는', '구', '구입', '구체적으로', '구토하다', '그', '그들', '그때', '그래', '그러한즉', '그런 까닭에', '그런즉', '그럼', '그럼에도 불구하고', '그렇게 함으로써', '그렇지', '그렇지 않다면', '그렇지않으면', '그리하여', '그만이다', '그에 따르는', '그위에', '그중에서', '그치지 않다', '근거로', '근거하여', '근거하여', '기대여', '기점으로', '기준으로', '기타', '까닭으로', '까악', '까지', '까지 미치다', '까지도', '꽈당', '끙끙', '나', '나다', '남들', '남짓', '너', '너희', '너희들', '네', '넷', '년', '논하지 않다', '놀라다', '누가 알겠는가', '누구', '는', '다른', '다른 방면으로', '다만', '다섯', '다소', '다수', '다시 말하자면', '다시말하면', '다음', '다음으로', '단지', '답다', '당신', '당장', '대로 하다', '대하면', '대하여', '대해 말하자면', '대해서', '댕그', '더구나', '더군다나', '더라도', '더불어', '더욱더', '더욱이는', '도달하다', '도착하다', '동시에', '동안', '돼다', '되다', '되어다', '된바에야', '된이상', '두번째로', '둘', '둥둥', '뒤따라', '뒤이어', '든간에', '들', '등', '등등', '딩동', '따라', '따라', '따라서', '따위', '따지지 않다', '딱', '때', '때가 되어', '

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in bul]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)
  score = float(model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [ ]:
sentiment_predict('너무 좋다')

72.72% 확률로 긍정 리뷰입니다.
